In [1]:
import pandas as pd
import numpy as np
#from sklearn import train_test_split
import sklearn.metrics
import dataclasses
import lightgbm as lgb
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining

In [2]:
train_processed_df = pd.read_pickle("inputs/train_processed.pkl")

In [3]:
# search_space = {

# }

# pbt_scheduler = PopulationBasedTraining(
#         time_attr='time_total_s',
#         metric='mean_accuracy',
#         mode='max',
#         perturbation_interval=600.0,
#         hyperparam_mutations={
#             "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
#             "alpha": lambda: random.uniform(0.0, 1.0),
#         ...
#         })
# tune.run( train_processed_df, scheduler=pbt_scheduler)

In [4]:
# from sklearn.model_selection import train_test_split
# X=train_processed_df.drop('diabetes_mellitus', axis=1)
# y=train_processed_df.diabetes_mellitus
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=314, stratify=y)
# train_set = lgb.Dataset(X_train, label=y_train)
# test_set = lgb.Dataset(X_test, label=y_test)

In [5]:
def LightGBMCallback(env):
    """Assumes that `valid_0` is the target validation score."""
    _, metric, score, _ = env.evaluation_result_list[0]
    tune.report(**{metric: score})

In [6]:
def train_diabetes(config):
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    #train_processed_df = pd.read_pickle("inputs/train_processed.pkl")
    data = train_processed_df.drop('diabetes_mellitus', axis=1)
    target = train_processed_df.diabetes_mellitus
    train_x, test_x, train_y, test_y = train_test_split(
        data, target, test_size=0.25)
    train_set = lgb.Dataset(train_x, label=train_y)
    test_set = lgb.Dataset(test_x, label=test_y)
    gbm = lgb.train(
        config,
        train_set,
        valid_sets=[test_set],
        verbose_eval=False,
        callbacks=[LightGBMCallback])
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    tune.report(
        mean_accuracy=sklearn.metrics.roc_auc_score(test_y, pred_labels),
        done=True)


In [36]:
# config = {
#         "objective": "binary",
#         #"metric": "binary_error",
#         "metric": "mean_accuracy",
#         "verbose": -1,
#         "boosting_type": tune.grid_search(["gbdt", "dart"]),
#         "num_leaves": tune.randint(10, 1000),
#         "learning_rate": tune.loguniform(1e-2, 1e-1)
#     }
# analysis = tune.run(
#         train_diabetes,
#         metric="auc",
#         mode="max",
#         config=config,
#         num_samples=2, #Number of times to sample from the hyperparameter space
#         #scheduler=ASHAScheduler()
#         local_dir="/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/logging",
#         scheduler=PopulationBasedTraining()
#         )

# print("Best hyperparameters found were: ", analysis.best_config)

2021-03-18 15:03:27,518	WARNING worker.py:1107 -- Warning: The actor ImplicitFunc has size 85149703 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-03-18 15:03:27,646	WARNING util.py:152 -- The `start_trial` operation took 0.858 s, which may be a performance bottleneck.


Trial name,status,loc,boosting_type,learning_rate,num_leaves
train_diabetes_17355_00000,RUNNING,,gbdt,0.0200876,439


2021-03-18 15:03:28,892	WARNING util.py:152 -- The `start_trial` operation took 0.679 s, which may be a performance bottleneck.
2021-03-18 15:03:30,083	WARNING util.py:152 -- The `start_trial` operation took 0.733 s, which may be a performance bottleneck.
2021-03-18 15:03:31,472	WARNING util.py:152 -- The `start_trial` operation took 0.765 s, which may be a performance bottleneck.
(pid=35848) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
(pid=35848)   warnings.warn('Overriding the parameters from Reference Dataset.')
(pid=35848) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
(pid=35848)   warnings.warn('{} in param dict is overridden.'.format(cat_alias))
(pid=35863) /Users/anitaclement/

nt/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/ray/tune/function_runner.py", line 576, in _trainable_func
(pid=35863)     output = fn()
(pid=35863)   File "<ipython-input-35-e67547cfe31f>", line 15, in train_diabetes
(pid=35863)   File "/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/engine.py", line 267, in train
(pid=35863)     evaluation_result_list=evaluation_result_list))
(pid=35863)   File "<ipython-input-17-c4e673d45b99>", line 3, in LightGBMCallback
(pid=35863) IndexError: list index out of range
(pid=35863) Exception in thread Thread-2:
(pid=35863) Traceback (most recent call last):
(pid=35863)   File "/Users/anitaclement/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=35863)     self.run()
(pid=35863)   File "/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/ray/t

TuneError: ('Trials did not complete', [train_diabetes_17355_00000, train_diabetes_17355_00001, train_diabetes_17355_00002, train_diabetes_17355_00003])

In [50]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

from ray import tune
from ray.tune.schedulers import ASHAScheduler


def LightGBMCallback(env):
    """Assumes that `valid_0` is the target validation score."""
    _, metric, score, _ = env.evaluation_result_list[0]
    tune.report(**{metric: score})


def train_diabetes(config):
    data = train_processed_df.drop('diabetes_mellitus', axis=1)
    target = train_processed_df.diabetes_mellitus
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(
        data, target, test_size=0.20)
    train_set = lgb.Dataset(train_x, label=train_y)
    test_set = lgb.Dataset(test_x, label=test_y)
    gbm = lgb.train(
        config,
        train_set,
        valid_sets=[test_set],
        verbose_eval=False,
        callbacks=[LightGBMCallback])
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    tune.report(
        #mean_accuracy=sklearn.metrics.accuracy_score(test_y, pred_labels),
        #binary_error=sklearn.metrics.accuracy_score(test_y, pred_labels),
        binary_error=sklearn.metrics.roc_auc_score(test_y, pred_labels),
        done=True)


if __name__ == "__main__":
    config = {
        "objective": "binary",
        "metric": "binary_error",
        "verbose": -1,
        #"boosting_type": tune.grid_search(["gbdt", "dart"]),
        "boosting_type": "gbdt",
        "num_leaves": tune.randint(10, 40),
        'min_child_samples': tune.randint(100, 400),
        'min_child_weight': tune.grid_search([1e-7,1e-6,1e-5, 1e-4, 1e-3, 1e-2,1e-1]),
        'subsample': tune.uniform(0.4, 0.6),
        'colsample_bytree': tune.uniform(0.2, 0.6),
        'reg_alpha': tune.grid_search([0, 1, 2, 5, 7, 10, 50]),
        'reg_lambda': tune.grid_search([0, 1, 5, 10, 20, 50]),
        'scale_pos_weight': tune.grid_search([1, 2, 3]),
        #"learning_rate": tune.loguniform(1e-2, 1e-1)
        "learning_rate": 0.01
    }

    analysis = tune.run(
        train_diabetes,
        metric="binary_error",
        #mode="min",
        mode="max",
        resources_per_trial={"cpu": 7, "gpu": 0},
        config=config,
        num_samples=1,
        #scheduler=ASHAScheduler())
        scheduler=PopulationBasedTraining(),
        local_dir="/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/logging")

    print("Best hyperparameters found were: ", analysis.best_config)

2021-03-18 16:08:41,721	WARNING worker.py:1107 -- Warning: The actor ImplicitFunc has size 85149680 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-03-18 16:08:41,862	WARNING util.py:152 -- The `start_trial` operation took 0.724 s, which may be a performance bottleneck.


Trial name,status,loc,colsample_bytree,min_child_samples,min_child_weight,num_leaves,reg_alpha,reg_lambda,scale_pos_weight,subsample
train_diabetes_344c8_00000,RUNNING,,0.283789,122,1e-07,36,0,0,1,0.46226


2021-03-18 16:08:42,848	WARNING util.py:152 -- The `start_trial` operation took 0.520 s, which may be a performance bottleneck.
2021-03-18 16:08:43,902	WARNING util.py:152 -- The `start_trial` operation took 0.630 s, which may be a performance bottleneck.
2021-03-18 16:08:45,154	WARNING util.py:152 -- The `start_trial` operation took 0.691 s, which may be a performance bottleneck.
2021-03-18 16:08:46,266	WARNING util.py:152 -- The `start_trial` operation took 0.590 s, which may be a performance bottleneck.
2021-03-18 16:08:47,293	WARNING util.py:152 -- The `start_trial` operation took 0.599 s, which may be a performance bottleneck.


Trial name,status,loc,colsample_bytree,min_child_samples,min_child_weight,num_leaves,reg_alpha,reg_lambda,scale_pos_weight,subsample
train_diabetes_344c8_00000,RUNNING,,0.283789,122,1e-07,36,0,0,1,0.46226
train_diabetes_344c8_00001,RUNNING,,0.430756,288,1e-06,16,0,0,1,0.435398
train_diabetes_344c8_00002,RUNNING,,0.318188,279,1e-05,38,0,0,1,0.482496
train_diabetes_344c8_00003,RUNNING,,0.512273,229,0.0001,16,0,0,1,0.589058
train_diabetes_344c8_00004,RUNNING,,0.347268,126,0.001,27,0,0,1,0.567391
train_diabetes_344c8_00005,RUNNING,,0.247092,291,0.01,29,0,0,1,0.427536


2021-03-18 16:08:48,221	WARNING util.py:152 -- The `start_trial` operation took 0.538 s, which may be a performance bottleneck.
2021-03-18 16:08:49,186	WARNING util.py:152 -- The `start_trial` operation took 0.543 s, which may be a performance bottleneck.
2021-03-18 16:08:54,267	WARNING worker.py:1107 -- The actor or task with ID ffffffffffffffffd398a33839d669fb961ecf3d01000000 cannot be scheduled right now. It requires {CPU: 1.000000} for placement, but this node only has remaining {0.000000/8.000000 CPU, 4.589844 GiB/4.589844 GiB memory, 1.000000/1.000000 node:192.168.0.4, 1.562500 GiB/1.562500 GiB object_store_memory}
. In total there are 0 pending tasks and 8 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.


KeyboardInterrupt: 